In [1]:
import pandas as pd
import pyTigerGraph as tg
from pyTigerGraph import gsql
conn = tg.TigerGraphConnection(graphname="Synthea")
shell = gsql.Gsql(conn, certNeeded=False)

In [2]:
print(shell.gsql('''
CREATE QUERY providerPatients(VERTEX<Provider> pvdr) FOR GRAPH Synthea{
  Start = {pvdr};
  patients = SELECT pats FROM Start:s -(providerSeesPatient)-> Patient:pats;
  PRINT patients;
}

INSTALL QUERY providerPatients
'''))

Trying version: v3_0_0
Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The query providerPatients has been added!
Start installing queries, about 1 minute ...
providerPatients query: curl -X GET 'http://127.0.0.1:9000/query/Synthea/providerPatients?pvdrName=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.

[=================================================================] 100% (1/1) 



In [3]:
print(shell.gsql('''
CREATE QUERY AverageVisits(/* Parameters here */) FOR GRAPH Synthea { 
  GroupByAccum<VERTEX<Patient> p, STRING name, SumAccum<INT> visitCount> @@patientVisit;
	MinAccum<INT> @@fewestVisits;
	MaxAccum<INT> @@mostVisits;
	AvgAccum @@avgVisits;
	
  patients = {Patient.*};
  tmp = SELECT
	        p 
       FROM patients:p - (patientVisit:e) -> Visit:v
       ACCUM @@patientVisit += (p, p.FirstName + " " + p.LastName -> p.outdegree()),
	           @@fewestVisits += p.outdegree(),
	           @@mostVisits += p.outdegree(),
	           @@avgVisits += p.outdegree();
  
    PRINT @@patientVisit;

    PRINT @@mostVisits;
    PRINT @@fewestVisits;
    PRINT @@avgVisits;
}

INSTALL QUERY AverageVisits
'''))

Trying version: v3_0_0
Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The query AverageVisits has been added!
Start installing queries, about 1 minute ...
AverageVisits query: curl -X GET 'http://127.0.0.1:9000/query/Synthea/AverageVisits'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.

[=================================================================] 100% (1/1) 



In [4]:
print(shell.gsql('''
CREATE QUERY PatientVisitsByProvider(VERTEX<Provider> pvdr) FOR GRAPH Synthea { 
#Define an accumulator
  SetAccum<VERTEX<Patient>> @@patientList;
  MapAccum<VERTEX<Patient>, INT> @@patientVisits;
  #Start with Provider
  providers = {pvdr};
  #Find all visits for that Provider
  visits = SELECT 
	      v 
	   FROM providers:p-(providerVisit:e)->Visit:v;
  #Display how many total visits that provider had
  PRINT visits.size();
  #Accumulate a set of patients, declaring as set will not permit duplicates
  patients = SELECT 
	            p 
	     FROM visits:v-(patientSeen:e)->Patient:p
	     ACCUM  @@patientList += p, @@patientVisits += ( p -> 1);
   #Iterate through set printing out relevant information 
   PRINT @@patientVisits;
}

INSTALL QUERY PatientVisitsByProvider
'''))

Trying version: v3_0_0
Connecting to localhost:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The query PatientVisitsByProvider has been added!
Start installing queries, about 1 minute ...
PatientVisitsByProvider query: curl -X GET 'http://127.0.0.1:9000/query/Synthea/PatientVisitsByProvider?providerName=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.

[=================================================================] 100% (1/1) 



In [4]:
params = {"pvdr": "e44f438a-601b-30fb-85-eb-bfaff8ddcb5ad"}

conn.runInstalledQuery("providerPatients", params)

GET http://localhost:9000/query/Synthea/providerPatients
http://localhost:9000/query/Synthea/providerPatients?pvdrName=Cedrick207+Runte676
[{'patients': []}]


[{'patients': []}]

In [22]:
res = conn.runInstalledQuery("AverageVisits")

In [23]:
res[1]

{'@@mostVisits': 256}

In [24]:
res[2]

{'@@fewestVisits': 2}

In [25]:
res[3]

{'@@avgVisits': 38.29841}

In [ ]:
params = {"pvdr": "e44f438a-601b-30fb-85-eb-bfaff8ddcb5ad"}
conn.runInstalledQuery("PatientVisitsByProvider", params)